## Youtube


### Trending

In [22]:
#Importing all needed libraries
from selenium import webdriver
import time
import os
import time
import requests
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

import json
from lxml import html
import re
import csv
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import (
    NoSuchElementException,
    TimeoutException,
    WebDriverException,
    )

In [28]:
WAIT_TIME = 10

# Chrome driver should be un
executable_path=os.path.join('C:/Program Files (x86)/Google/Chrome/Application/chromedriver')

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--disable-notifications')

# 1-Allow, 2-Block, 0-default
preferences = {
    "profile.default_content_setting_values.notifications" : 2,
    "profile.default_content_setting_values.location": 2,
    # We don't need images, only the URLs.
    "profile.managed_default_content_settings.images": 2,
    }
options.add_experimental_option("prefs", preferences)


browser = webdriver.Chrome(
    executable_path=executable_path,
    chrome_options=options,
    )
browser.wait = WebDriverWait(browser, WAIT_TIME)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: use options instead of chrome_options


In [29]:
#Opening the browser and getting the url
#browser = webdriver.Chrome()
url = "https://www.youtube.com/feed/trending"
browser.get(url)
#wait 5 seconds to load
time=5

### Inspect source coude

![Inspecting_source_code](youtube_1.png)

![Inspecting_source_code](youtube_2.png)

### Identify target elements to be scrapped

Copy the source code on the notepad and search the specific components of your interest. The title and the URL will be retrieved directly from the browser (this is, without parsing to beautifulsoup). This can be done because these objects are quite straight forward.

#### Title (Hay una ligera imprecisión, pregunta)
![Title](youtube_3.png)

#### URL
![URL](youtube_4.png)

In [30]:
#getting trending video urls and titles
titles=[i.text for i in browser.find_elements_by_id('video-title')]
urls=[i.get_attribute('href') for i in browser.find_elements_by_css_selector('a#video-title')] #<a> tag with video title id

#### Duration

Duration or views are not straightfoward. We need to apply some regex functions to parse correctly views values. As an example, try to look for a known value and identify the token.

![Duration](youtube_5.png)

![Duration](youtube_6.png)

In [34]:
#getting duration and cleaning data
response = browser.page_source
page = BeautifulSoup(response,"html.parser")
duration=[i.get_text().replace("\n", "").strip() for i in  page.find_all("span", class_="style-scope ytd-thumbnail-overlay-time-status-renderer")]

#getting views and cleaning data
#regex find meaning:find everything with no white space "\S" then numbers then everything ".+" and str(views) 
all_labels=[i.get_attribute('aria-label') for i in browser.find_elements_by_css_selector('a#video-title')]

In [38]:
#getting duration and cleaning data
response = browser.page_source
page = BeautifulSoup(response,"html.parser")
duration=[i.get_text().replace("\n", "").strip() for i in  page.find_all("span", class_="style-scope ytd-thumbnail-overlay-time-status-renderer")]

#getting views and cleaning data
#regex find meaning:find everything with no white space "\S" then numbers then everything ".+" and str(views) 
all_labels=[i.get_attribute('aria-label') for i in browser.find_elements_by_css_selector('a#video-title')]
view=[i[0] for i in [re.findall("\S+[0-9]+.visualizaciones",i) for i in all_labels]] 
views=[i.replace("visualizaciones", "")for i in view]

username=[i.text for i in browser.find_elements_by_xpath('//*[@id="byline"]/a')]

In [40]:
df=pd.DataFrame(list(zip(titles,urls,duration, views)), columns=['Title','Url', 'Duration', 'Views'])
df.head(20)

,Title,Url,Duration,Views
0,EL COLCHÓN QUE SE HUNDE (Broma telefónica),https://www.youtube.com/watch?v=JsBerCq5PXo,17:41,3.691.306
1,POSEIDÓN | Destripando la Historia | Canción,https://www.youtube.com/watch?v=2-VTUyI3w7U,2:52,5.111.568
2,"Farruko, Ozuna, Lunay - Nadie (Remix - Officia...",https://www.youtube.com/watch?v=BRzwi2-P1cg,4:33,2.517.553
3,Rels B - ORGULLO (Videoclip Oficial),https://www.youtube.com/watch?v=jkfmFAdrbxg,3:06,931.687
4,Natos - TRANQUILITO ft. Alejandro Deese (Lyric...,https://www.youtube.com/watch?v=fl1VDm719IQ,3:04,136.594
5,LA RESISTENCIA - Entrevista a Gerard Piqué | #...,https://www.youtube.com/watch?v=7b29bya_u2A,20:51,749.555
6,OT CASTING LAS PALMAS DE GRAN CANARIA | FASE 1...,https://www.youtube.com/watch?v=uttZfh9UWQg,3:55:34,76.810
7,REACCIONANDO AL VÍDEO MÁS GRACIOSO QUE HE VIST...,https://www.youtube.com/watch?v=gTwZC-BrOKM,17:19,246.301
8,Harry Styles - Lights Up (Official Video),https://www.youtube.com/watch?v=9NZvM1918_E,2:56,3.930.107
9,"TINI, Sebastián Yatra - Oye",https://www.youtube.com/watch?v=azfKhDMIrZo,3:24,1.172.427


In [41]:
#closing the browser
browser.close()

In [42]:
df.to_csv('LeireAlegriaPractica1.csv',index=False, header=True)